# Comparación Presupuestos 2020-2021

Noviembre, 2020

In [1]:
import sqlite3
import pandas as pd
import json

In [2]:
DB = "../../prep/budget.db"
conn = sqlite3.connect(DB)

In [10]:
sql_stmt = """
    SELECT office, unit, SUBSTR(object, 1, 2) AS object, SUM(amount) AS amount
        FROM budget 
        WHERE year={} AND moment='{}'
        GROUP BY office, unit, object
"""

In [16]:
def adjust_office_name(name):
    return name\
        .replace('RAMO DE LA ', '') \
        .replace('RAMO DE', '') \
        .replace(' DE LA REPUBLICA', '') \
        .replace('HOSPITAL NACIONAL', 'HOSP') \
        .replace('INSTITUTO', 'INST') \
        .replace('SALVADORE¥O', 'SALV')

In [17]:
def add_office_names(ds):
    offices = pd.read_sql("SELECT * FROM office", conn)
    offices['office_name'] = offices['office_name'].apply(adjust_office_name)
    heads = pd.read_sql('SELECT * FROM object', conn)
    ds = ds.merge(offices, on='office').merge(heads, on='object')
    a_unit = '10'
    b_unit = '20'
    for i, r in ds.iterrows():
        if r.office == '0700':
            if r.unit >= a_unit and r.unit <= b_unit:
                ds.loc[i, 'office_name'] = 'TRANSFERENCIAS Y OBLIGACIONES GENERALES'
            elif r.unit > b_unit:
                ds.loc[i, 'office_name'] = 'DEUDA PÚBLICA'
    ds = ds.groupby(['office', 'office_name', 'level', 'object', 'object_name']).sum()['amount'].reset_index()
    return ds

In [18]:
p2021 = add_office_names(pd.read_sql(sql_stmt.format(2021, 'PR'), conn))
p2020 = add_office_names(pd.read_sql(sql_stmt.format(2020, 'AP'), conn))
comp = p2020.merge(p2021, on=['office', 'office_name',  'level', 'object', 'object_name'])
comp.rename(columns={'amount_x': 'enacted_2020', 'amount_y': 'proposed_2021'}, inplace=True)
comp['diff'] = comp['proposed_2021'] - comp['enacted_2020']
comp.to_csv('comparative.csv', index=False)

In [21]:
comp.head()

,office,office_name,level,object,object_name,enacted_2020,proposed_2021,diff
0,0200,CORTE DE CUENTAS,CG,51,Remuneraciones,34417115.0,34413575.0,-3540.0
1,0200,CORTE DE CUENTAS,CG,54,Adquisiciones de Bienes y Servicios,5627190.0,5189101.0,-438089.0
2,0200,CORTE DE CUENTAS,CG,55,Gastos Financieros y Otros,3062550.0,3205150.0,142600.0
3,0200,CORTE DE CUENTAS,CG,56,Transferencias Corrientes,13000.0,13000.0,0.0
4,0200,CORTE DE CUENTAS,CG,61,Inversiones en Activos Fijos,1152720.0,889030.0,-263690.0


In [19]:
value_fields = ['enacted_2020', 'proposed_2021', 'diff']

def filter_offices(level):
    return comp[comp.level == level].groupby(['office', 'office_name', 'level'])\
        .sum()[value_fields]\
        .reset_index()\
        .rename(columns={'office': 'code', 'office_name': 'name'})\
        .to_dict('records')

rec = {
    'name': 'root',
    'cg': filter_offices('CG'),
    'de': filter_offices('DE'),
}

for f in ['cg', 'de']:
    
    rec['head_' + f] = comp[comp.level == f.upper()].groupby(['object', 'object_name'])\
        .sum()[value_fields]\
        .reset_index()\
        .rename(columns={'object': 'code', 'object_name': 'name'})\
        .to_dict('records')
    for r in rec[f]:
        r['children'] = comp[comp.office_name ==  r['name']]\
            [['object', 'object_name'] + value_fields]\
            .rename(columns={'object': 'code', 'object_name': 'name'})\
            .to_dict('records')
    for field in value_fields:
        rec[field] = sum([value[field] for value in rec[f]])

with open('comparative.json', 'w') as f:
    json.dump(rec, f)

In [20]:
rec.keys()

dict_keys(['name', 'cg', 'de', 'head_cg', 'enacted_2020', 'proposed_2021', 'diff', 'head_de'])

In [49]:
comp.level.unique()

array(['CG', 'DE'], dtype=object)